# Sunshine Heavy Industries Constraint Solver

Currently only solves for basic numerical constraints + fuel & heatsink adjacency

todo: armor requirements, cloaking area constraints, energy area constraints.

In [1]:
from json import loads
import numpy as np
import pandas as pd
from pyomo.environ import *

costs = None
with open("costs.json") as f:
    costs = list(loads(f.read()).items())

print(f"Parts in DB: {len(costs)}")

Parts in DB: 108


In [2]:
def prop_vector(prop_name, default=0):
    return np.array([data.get(prop_name, default) for _, data in costs])

names = [n for n, _ in costs]
hitboxes = [np.array(data.get('hitbox', np.ones([data.get('width'),data.get('height')]))) for _, data in costs]
area = np.array([np.count_nonzero(hb) for hb in hitboxes])

borders = np.array([[not x in ob for x in range(4)] for ob in prop_vector('mustBeUnobstructed', [])])

perimeter = \
    prop_vector('height') * borders[:,0] + \
    prop_vector('width') * borders[:, 1] + \
    prop_vector('height') * borders[:, 2] + \
    prop_vector('width') * borders[:, 3]

data = pd.DataFrame({
    'part': names,
    'cost': area * prop_vector('cost'),
    'energy': area * prop_vector('energy'),
    'fuel': area * prop_vector('fuel'),
    'firepower': area * prop_vector('firepower'),
    'passenger': area * prop_vector('quarters'),
    'command': area * prop_vector('command'),
    'cargo': area * prop_vector('cargo'),
    'fuelAdjacency': prop_vector('pump') * perimeter - prop_vector('mustTouchFuelPump'),
    'sinkAdjacency': prop_vector('heatsink') * perimeter - prop_vector('heat'),
})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

data.set_index('part', inplace=True)
data



,cost,energy,fuel,firepower,passenger,command,cargo,fuelAdjacency,sinkAdjacency
part,,,,,,,,,
antenna_1x2,100.0,0.0,0,0,0,0,0,0,0
antenna_1x4,400.0,0.0,0,0,0,0,0,0,0
antenna_1x5,300.0,0.0,0,0,0,0,0,0,0
bunk_1x1,20.0,0.0,0,0,1,0,0,0,0
bunk_2x1,40.0,0.0,0,0,2,0,0,0,0
bunk_4x1,80.0,0.0,0,0,4,0,0,0,0
bunk_4x2,160.0,0.0,0,0,8,0,0,0,0
bunk_norad_4x2,320.0,0.0,0,0,8,0,0,0,0
cab_2x2,400.0,0.0,0,0,4,4,0,0,0


In [4]:
model = ConcreteModel()
model.x = Var(names, domain=NonNegativeIntegers)
model.total_cost = Objective(
    expr = sum(data.loc[i].cost *model.x[i] for i in names),
    sense = minimize
)
model.fuel = Constraint(
    expr = sum(data.loc[i].fuel * model.x[i] for i in names) >= 12
)
model.energy = Constraint(
    expr = sum(data.loc[i].energy * model.x[i] for i in names) >= 10
)
model.firepower = Constraint(
    expr = sum(data.loc[i].firepower * model.x[i] for i in names) >= 53
)
model.passenger = Constraint(
    expr = sum(data.loc[i].passenger * model.x[i] for i in names) >= 10
)
model.command = Constraint(
    expr = sum(data.loc[i].command * model.x[i] for i in names) >= 10
)
model.cargo = Constraint(
    expr = sum(data.loc[i].cargo * model.x[i] for i in names) >= 10
)
model.fuelAdjacency = Constraint(
    expr = sum(data.loc[i].fuelAdjacency * model.x[i] for i in names) >= 0
)
model.sinkAdjacency = Constraint(
    expr = sum(data.loc[i].sinkAdjacency * model.x[i] for i in names) >= 0
)

solver=SolverFactory('glpk', executable="/usr/local/bin/glpsol")
results = solver.solve(model)
print(results)
model.display()


res = []
for i in model.x:
    if(model.x[i].value > 0):
        res.append([str(model.x[i])[2:-1], int(model.x[i].value)])
        
df = pd.DataFrame(res, columns=['Part','Qty'])
df.style.hide_index()


Problem: 
- Name: unknown
  Lower bound: 5090.0
  Upper bound: 5090.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 109
  Number of nonzeros: 125
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 143
      Number of created subproblems: 143
  Error rc: 0
  Time: 0.014417886734008789
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Model unknown

  Variables:
    x : Size=108, Index=x_index
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
                antenna_1x2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                antenna_1x4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                antenna_1x5 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                   bunk_1x1 :     0 :  10.0 :  None : False : False : NonNegativeIntegers
                  

Part,Qty
bunk_1x1,10
cannon_2x1,1
cannon_2x3,1
cannon_5x1,1
cargo_1x1,10
cockpit_1x2,1
cockpit_1x4,2
generator_2x2,1
intake_3x2,1
pump_1x1,1
